In [ ]:
# Cell: dependency_checker
"""
Dependencies: None (root cell)
Provides: Dependency checking functionality
"""
import logging
from typing import Dict, Set, Any
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class DependencyChecker:
    def __init__(self):
        self.loaded_cells = set()
        self.load_times = {}
        self.dependency_map = {
            'dependency_checker': set(),  # No dependencies
            'notebook_monitor': {'dependency_checker'},
            'imports': {'dependency_checker', 'notebook_monitor'},
            'global_constants': {'imports'},
            'environment_config': {'global_constants', 'imports'},
            'snowflake_utils': {'environment_config', 'global_constants', 'imports'},
            'model_scoring': {'snowflake_utils', 'global_constants'},
            'score_validation': {'model_scoring', 'global_constants'},
            'scoring_report': {'score_validation', 'global_constants'}
        }

    def register_cell(self, cell_name: str):
        """Register a cell as loaded"""
        self.loaded_cells.add(cell_name)
        self.load_times[cell_name] = datetime.now()
        logger.info(f"Registered cell: {cell_name}")
        print(f"Registered cell: {cell_name}")

    def check_dependencies(self, cell_name: str) -> bool:
        """Check if all dependencies for a cell are loaded"""
        if cell_name not in self.dependency_map:
            logger.warning(f"Unknown cell: {cell_name}")
            return False
        
        missing = self.dependency_map[cell_name] - self.loaded_cells
        if missing:
            logger.error(f"Missing dependencies for {cell_name}: {missing}")
            return False
        return True

    def show_dependency_tree(self):
        """Display the dependency tree in a readable format"""
        print("\nDependency Tree:")
        print("=" * 50)
        for cell in self.dependency_map:
            deps = self.dependency_map[cell]
            status = "✓" if self.check_dependencies(cell) else "✗"
            loaded = "✓" if cell in self.loaded_cells else "✗"
            print(f"{status} {cell} [{loaded}]")
            if deps:
                for dep in deps:
                    dep_loaded = "✓" if dep in self.loaded_cells else "✗"
                    print(f"  ├── {dep_loaded} {dep}")
            print("  │")

# Initialize the dependency checker
dep_checker = DependencyChecker()
dep_checker.register_cell('dependency_checker')

# Show initial status
logger.info("Dependency checker initialized")
dep_checker.show_dependency_tree()

In [ ]:
# Cell: notebook_monitor
"""
Dependencies: dependency_checker
Provides: Enhanced monitoring with memory usage and operation counts
"""
import time
import pandas as pd
import psutil
import os
from datetime import datetime
from typing import Optional, Dict, List, Any

class ScoringMonitor:
    def __init__(self):
        if not dep_checker.check_dependencies('notebook_monitor'):
            print("Warning: Dependencies not met for notebook monitor")
        
        self.execution_logs = []
        self.current_cell = None
        self.start_time = None
        self.end_time = None
        self.operation_counts = {}
        self.scoring_metrics = {}
        self.context = []
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)

    def add_context(self, context: str) -> None:
        """Add context to the current monitoring"""
        self.context.append(context)
        
    def remove_context(self) -> None:
        """Remove the most recent context"""
        if self.context:
            self.context.pop()
    
    def _get_memory_usage(self):
        """Get current memory usage in MB"""
        process = psutil.Process(os.getpid())
        return process.memory_info().rss / 1024 / 1024
    
    def start(self, operation_name: str) -> None:
        """Start monitoring an operation"""
        if self.context:
            operation_name = f"{operation_name}_{'_'.join(self.context)}"
        
        self.current_cell = {
            'operation': operation_name,
            'start_time': datetime.now(),
            'status': 'running',
            'duration_seconds': 0,
            'start_memory_mb': self._get_memory_usage(),
            'records_processed': 0,
            'error': None
        }
        self.start_time = time.time()
    
    def update_metrics(self, metrics: Dict[str, Any]) -> None:
        """Update scoring metrics"""
        if self.current_cell:
            self.current_cell.update(metrics)
    
    def end(self) -> None:
        """End monitoring current operation"""
        if self.current_cell:
            self.end_time = time.time()
            duration = self.end_time - self.start_time
            end_memory = self._get_memory_usage()
            
            self.current_cell.update({
                'end_time': datetime.now(),
                'status': 'completed',
                'duration_seconds': round(duration, 2),
                'end_memory_mb': end_memory,
                'memory_change_mb': round(end_memory - self.current_cell['start_memory_mb'], 2)
            })
            
            self.execution_logs.append(self.current_cell.copy())
            self.current_cell = None
    
    def show_summary(self) -> pd.DataFrame:
        """Show execution summary"""
        if not self.execution_logs:
            return pd.DataFrame()
        
        try:
            # Create DataFrame from logs
            df = pd.DataFrame(self.execution_logs)
            
            # Format timestamps
            for col in ['start_time', 'end_time']:
                if col in df.columns:
                    df[col] = pd.to_datetime(df[col]).dt.strftime('%Y-%m-%d %H:%M:%S')
            
            # Add scoring specific metrics
            total_duration = df['duration_seconds'].sum()
            total_records = df['records_processed'].sum()
            
            print("\nScoring Summary:")
            print(f"Total Duration: {total_duration:.2f} seconds")
            print(f"Total Records Processed: {total_records:,}")
            if total_duration > 0:
                print(f"Average Processing Rate: {total_records/total_duration:.2f} records/second")
            
            return df
        
        except Exception as e:
            print(f"Error generating summary: {type(e).__name__} - {str(e)}")
            print(traceback.format_exc())
            return pd.DataFrame()
    
    def log_error(self, error: Exception) -> None:
        """Log an error for the current operation"""
        if self.current_cell:
            self.current_cell.update({
                'status': 'error',
                'error': f"{type(error).__name__}: {str(error)}",
                'end_memory_mb': self._get_memory_usage()
            })
            self.end()

# Initialize monitor
monitor = ScoringMonitor()
dep_checker.register_cell('notebook_monitor')

In [ ]:
# Cell: imports
monitor.start('imports')
try:
    """
    Dependencies: dependency_checker, notebook_monitor
    Provides: All required package imports for the scoring notebook
    """
    # Standard libraries
    import logging
    from typing import Dict, Set, Any, List, Optional, Tuple
    from datetime import datetime
    import time
    import traceback
    import json
    import gc
    import pickle
    import base64
    import copy
    import joblib
    import io

    
    # Snowpark
    from snowflake.snowpark.functions import col, when
    import snowflake.snowpark.functions as F
    from snowflake.snowpark import Session

    def get_active_session() -> Session:
        return Session.get_active_session()
    
    # Models
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.svm import SVC
    from sklearn.neural_network import MLPClassifier
    from sklearn.impute import SimpleImputer
    from sklearn.preprocessing import StandardScaler

    # Scientific computing
    import pandas as pd
    import numpy as np

    # Visualization
    import matplotlib.pyplot as plt
    import seaborn as sns

    # Configure logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logger = logging.getLogger(__name__)

finally:
    monitor.end()
dep_checker.register_cell('imports')

In [ ]:
# Cell: global_constants
monitor.start('global_constants')
try:
    """
    Dependencies: imports
    Provides: Global constants for the scoring notebook
    """
    # Department and Model Configuration
    DEPARTMENT = 'UF'
    MODEL_TYPE = 'MAJOR_GIFT'
    
    # Database configuration
    SCHEMA = "UFF_MODEL"
    SOURCE_TABLE = "UF_MAJOR_GIFT_VIEWS_COMBO"
    
    # Scoring parameters
    CHUNK_SIZE = 150000   # Size of processing chunks
    LIMIT = 1500000
    
    # Model Selection Weights
    MODEL_SELECTION_WEIGHTS = {
        'PRIMARY': {
            'F1_MEAN': 0.30,
            'ROC_AUC_MEAN': 0.30,
            'MCC_MEAN': 0.20,
            'BALANCED_ACCURACY_MEAN': 0.20
        },
        'SECONDARY': {
            'PERFORMANCE': 0.40,
            'PRECISION_RECALL_BALANCE': 0.30,
            'STABILITY': 0.30
        }
    }

    OUTPUT_COLUMNS = {
        'required': [
            'ID',
            'AFFINITY_SCORE',
            'AFFINITY_GRADE',
            'PROBABILITY',
            'SCORED_DATE'
        ],
        'optional': [
            'MODEL_NAME',
            'MODEL_VERSION',
            'FEATURE_COUNT',
            'CONFIDENCE'
        ]
    }
    
    VALIDATION_THRESHOLDS = {
        'min_score_count': 1000,
        'max_null_percentage': 0.01,
        'grade_distribution_bounds': {
            'A': (0.001, 0.01),
            'B': (0.01, 0.05),
            'C': (0.05, 0.15),
            'D': (0.15, 0.30),
            'E': (0.40, 0.80)
        }
    }

    PROBABILITY_THRESHOLDS = {
        'A': 0.995,
        'B': 0.980,
        'C': 0.950,
        'D': 0.850,
        'E': 0.000
    }
    
    GRADE_THRESHOLDS = {
        'A': 99.5,
        'B': 98.0,
        'C': 95.0,
        'D': 85.0,
        'E': 0.0
    }

    # Grade configuration
    GRADING_METHOD = 'PERCENTILE'  # or PERCENTILE or FIXED
    
    # Percentile-based grading configuration
    PERCENTILE_GRADE_CONFIG = {
        'A': 99,    # Top 1% (99th percentile and above)
        'B': 95,    # 95-99th percentile  
        'C': 90,    # 90-95th percentile
        'D': 85,    # 80-90th percentile # Changed from 80 to 85
        'E': 0      # Bottom 80%
    }
    
    
    FIXED_GRADE_RANGES = [
        ('A', 99.50, float('inf')),
        ('B', 98.00, 99.49),
        ('C', 95.00, 97.99),
        ('D', 85.00, 94.99),
        ('E', float('-inf'), 85.00)
    ]
        
    def get_table_name(base_name: str, timestamp: Optional[str] = None) -> str:
        """Generate standardized table names"""
        name = f"{DEPARTMENT}_{MODEL_TYPE}_{base_name}"
        if timestamp:
            name = f"{name}_{timestamp}"
        return name

    # Define standard table names
    TABLE_NAMES = {
        'SOURCE': SOURCE_TABLE,
        'MODEL_RESULTS': get_table_name('MODEL_PERFORMANCE'),
        'MODELS': get_table_name('MODELS'),
        'FEATURES': get_table_name('FEATURE_IMPORTANCE'),
        'SCORES': get_table_name('AFFINITY_SCORES'),
        'METADATA': get_table_name('SCORING_METADATA')
    }

    TABLE_SCHEMAS = {
        'SCORES': """(
            ID VARCHAR,
            PROBABILITY FLOAT,
            AFFINITY_SCORE FLOAT,
            AFFINITY_GRADE VARCHAR,
            SCORED_DATE TIMESTAMP_NTZ,
            CREATED_AT TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP()
        )""",
        'METADATA': """(
            SCORING_RUN_ID VARCHAR,
            MODEL_NAME VARCHAR,
            FEATURE_SAMPLE VARCHAR,
            N_FEATURES NUMBER,
            F1_SCORE FLOAT,
            ROC_AUC FLOAT,
            RECORDS_SCORED NUMBER,
            SCORING_START TIMESTAMP_NTZ,
            WHERE_CLAUSE VARCHAR,
            GRADE_DISTRIBUTION VARIANT,
            FEATURES_USED VARIANT
        )"""
    }

    print("\nConfiguration Settings:")
    print("=" * 50)
    
    print("\nTable References:")
    for key, value in TABLE_NAMES.items():
        print(f"{key}: {value}")
        
    print("\nModel Selection Weights:")
    print("\nPrimary Weights:")
    for metric, weight in MODEL_SELECTION_WEIGHTS['PRIMARY'].items():
        print(f"{metric}: {weight:.2f}")
    print("\nSecondary Weights:")
    for aspect, weight in MODEL_SELECTION_WEIGHTS['SECONDARY'].items():
        print(f"{aspect}: {weight:.2f}")
        
    print("\nAffinity Grade Thresholds:")
    for grade, threshold in GRADE_THRESHOLDS.items():
        print(f"Grade {grade}: >= {threshold}")

finally:
    monitor.end()
dep_checker.register_cell('global_constants')

In [ ]:
# Cell: environment_config
monitor.start('environment_config')
try:
    class EnvironmentConfig:
        # Static configurations that don't change between environments
        SCHEMA = SCHEMA
        TABLES = {
            'SOURCE': TABLE_NAMES['SOURCE'],
            'MODEL_RESULTS': TABLE_NAMES['MODEL_RESULTS'],
            'MODELS': TABLE_NAMES['MODELS'],
            'FEATURES': TABLE_NAMES['FEATURES'],
            'SCORES': TABLE_NAMES['SCORES'],
            'METADATA': TABLE_NAMES['METADATA']
        }
        
        # Environment configurations
        ENVIRONMENTS = {
            'test': {
                'database': 'PRE_PRODUCTION',
                'schema': SCHEMA,
                'warehouse': 'ANALYSIS'
            },
            'prod': {
                'database': 'PRODUCTION',
                'schema': SCHEMA,
                'warehouse': 'ANALYSIS'
            }
        }
        
        def __init__(self):
            self.current_env = 'test'  # default to test environment
            self.session = None
            
            # Validate table configurations
            required_tables = {'SOURCE', 'MODEL_RESULTS', 'MODELS', 'FEATURES', 'SCORES', 'METADATA'}
            missing_tables = required_tables - set(self.TABLES.keys())
            if missing_tables:
                raise ValueError(f"Missing required table configurations: {missing_tables}")

        def set_session(self, session):
            """Set the Snowflake session"""
            self.session = session
            self._apply_environment()
        
        def _apply_environment(self):
            """Apply environment settings to Snowflake session"""
            if not self.session:
                raise ValueError("Session not initialized")
                
            env = self.ENVIRONMENTS[self.current_env]
            try:
                self.session.sql(f"USE DATABASE {env['database']}").collect()
                self.session.sql(f"USE SCHEMA {env['schema']}").collect()
                self.session.sql(f"USE WAREHOUSE {env['warehouse']}").collect()
            except Exception as e:
                print(f"Error setting environment: {str(e)}")
                raise

        def get_schema_db_name(self) -> str:
            """Get database.schema"""
            env = self.ENVIRONMENTS[self.current_env]
            return f"{env['database']}.{env['schema']}"
            
        def show_environment_status(self):
            """Display current environment settings"""
            if not self.session:
                raise ValueError("Session not initialized")
                
            border = "=" * 60
            print(border)
            print(f"{'ENVIRONMENT STATUS':^60}")
            print(border)
            print(f"ENVIRONMENT: {self.current_env.upper()}")
            
            try:
                current_settings = self.session.sql("""
                SELECT 
                    CURRENT_DATABASE() as database,
                    CURRENT_SCHEMA() as schema,
                    CURRENT_WAREHOUSE() as warehouse
                """).collect()
                
                print(f"""
                DATABASE:  {current_settings[0]['DATABASE']}
                SCHEMA:    {current_settings[0]['SCHEMA']}
                WAREHOUSE: {current_settings[0]['WAREHOUSE']}
                """)
                print(border)
            except Exception as e:
                print(f"Error getting current settings: {str(e)}")
                raise

finally:
    monitor.end()
dep_checker.register_cell('environment_config')

In [ ]:
# Cell: snowflake_utils
monitor.start('snowflake_utils')
try:
    class ScoringDatabaseManager:
        def __init__(self, config: EnvironmentConfig):
            self.config = config
            self.session = config.session
            self.max_retries = 3
            self.retry_delay = 5  # seconds

        def execute_with_retry(self, operation_name: str, operation, *args, **kwargs):
            """Execute operation with retry logic"""
            monitor.start(f'execute_{operation_name}')
            try:
                for attempt in range(self.max_retries):
                    try:
                        result = operation(*args, **kwargs)
                        return result
                    except Exception as e:
                        if attempt < self.max_retries - 1:
                            print(f"Error in {operation_name}, retrying... (Attempt {attempt + 1}/{self.max_retries})")
                            time.sleep(self.retry_delay * (attempt + 1))
                            self.session = get_active_session()
                            self.config.set_session(self.session)
                        else:
                            raise
            finally:
                monitor.end()

        def load_data_for_scoring(self, features: List[str],
                                 where_clause: Optional[str] = None,
                                 limit: Optional[int] = None) -> pd.DataFrame:
            """Load and preprocess data for scoring, applying same encoding as training"""
            monitor.start('load_scoring_data')
            try:
                # First, load a sample to identify data types (same as training)
                sample_query = f"""
                SELECT *
                FROM {self.config.get_schema_db_name()}.{TABLE_NAMES['SOURCE']}
                LIMIT 1000
                """
                sample_df = self.execute_with_retry('load_sample', 
                                                  lambda: self.session.sql(sample_query).to_pandas())
                
                # Use SAME logic as training to identify categorical columns
                categorical_columns = sample_df.select_dtypes(include=['object']).columns.tolist()
                print(f"Identified categorical columns by dtype: {categorical_columns}")
                
                # Now identify which of our features come from these categorical columns
                original_features = set(['DONOR_ID'])
                categorical_prefixes = set()
                
                for feature in features:
                    # Check if this feature could be from a categorical column
                    feature_is_categorical = False
                    for cat_col in categorical_columns:
                        if feature.startswith(cat_col + '_'):
                            categorical_prefixes.add(cat_col)
                            feature_is_categorical = True
                            break
                    
                    if not feature_is_categorical:
                        original_features.add(feature)
                
                print(f"Categorical prefixes needed: {categorical_prefixes}")
                print(f"Original features needed: {original_features}")
                
                # Load original columns (categorical + numeric + DONOR_ID)
                columns_to_load = list(original_features | categorical_prefixes)
                feature_list = ', '.join([f'"{f}"' for f in columns_to_load])
                
                query = f"""
                SELECT {feature_list}
                FROM {self.config.get_schema_db_name()}.{TABLE_NAMES['SOURCE']}
                """
                if where_clause:
                    query += f" WHERE {where_clause}"
                if limit:
                    query += f" LIMIT {limit}"
                    
                print(f"Loading original data columns: {columns_to_load}")
                df = self.execute_with_retry('load_data', lambda: self.session.sql(query).to_pandas())
                
                # Apply same preprocessing as training
                processed_df = df[['DONOR_ID']].copy()  # Start with DONOR_ID
                
                # Add original numeric features
                for feature in original_features:
                    if feature != 'DONOR_ID' and feature in df.columns:
                        processed_df[feature] = df[feature]
                
                # Apply one-hot encoding for categorical features
                for cat_col in categorical_prefixes:
                    if cat_col in df.columns:
                        print(f"Processing categorical column: {cat_col}")
                        # Apply same cleaning as training
                        cleaned_series = df[cat_col].astype(str).apply(
                            lambda x: x.replace(' ', '').replace("'", "")
                        ).replace("-", "")
                        
                        # Apply one-hot encoding with same parameters as training
                        dummies = pd.get_dummies(
                            cleaned_series,
                            prefix=cat_col,
                            dummy_na=False,
                            drop_first=False
                        )
                        
                        # Apply same column name standardization as training
                        dummies.columns = [
                            col_name.upper().replace(' ', '_').replace('-', '_').replace('.', '_')
                            for col_name in dummies.columns
                        ]
                        
                        # Only keep columns that existed during training
                        expected_columns = [f for f in features if f.startswith(cat_col + '_')]
                        training_columns = set(expected_columns)
                        scoring_columns = set(dummies.columns)
                        
                        # Keep only columns that were in training
                        valid_columns = list(training_columns & scoring_columns)
                        if valid_columns:
                            dummies = dummies[valid_columns]
                            print(f"Kept {len(valid_columns)} training columns for {cat_col}")
                        
                        # Log unseen categories for monitoring
                        unseen_columns = scoring_columns - training_columns
                        if unseen_columns:
                            print(f"WARNING: Found unseen categories for {cat_col}: {unseen_columns}")
                            print("These will be ignored in scoring")
                        
                        # Add to processed dataframe
                        processed_df = pd.concat([processed_df, dummies], axis=1)
                
                # Ensure we have all required features, add missing ones as zeros
                missing_features = set(features) - set(processed_df.columns)
                if missing_features:
                    print(f"Adding missing features as zeros: {missing_features}")
                    for feature in missing_features:
                        processed_df[feature] = 0.0
                
                # Reorder columns to match expected feature order
                final_columns = ['DONOR_ID'] + [f for f in features if f != 'DONOR_ID']
                processed_df = processed_df[final_columns]
                
                print(f"Final processed data shape: {processed_df.shape}")
                print(f"Final columns: {processed_df.columns.tolist()}")
                
                return processed_df
                
            finally:
                monitor.end()        		
  

        def save_scores(self, df: pd.DataFrame, timestamp: str = None) -> None:
            """Save scoring results"""
            monitor.start('save_scores')
            try:
                if 'SCORES' not in TABLE_NAMES:
                    raise ValueError("SCORES table name not found in TABLE_NAMES")
                
                if 'required' not in OUTPUT_COLUMNS:
                    raise ValueError("Required columns not defined in OUTPUT_COLUMNS")
                
                missing_cols = set(OUTPUT_COLUMNS['required']) - set(df.columns)
                if missing_cols:
                    raise ValueError(f"Missing required columns: {missing_cols}")
                
                table_name = TABLE_NAMES['SCORES']
                full_name = f"{self.config.get_schema_db_name()}.{table_name}"
                
                snowpark_df = self.session.create_dataframe(df)
                snowpark_df.write.save_as_table(full_name, mode='overwrite')
                
                print(f"Overwritten scores to: {full_name}")
                
            finally:
                monitor.end()

        def save_scoring_metadata(self, metadata: Dict[str, Any]) -> None:
            """Save metadata about scoring run"""
            monitor.start('save_metadata')
            try:
                metadata_df = pd.DataFrame([metadata])
                table_name = TABLE_NAMES['METADATA']
                full_name = f"{self.config.get_schema_db_name()}.{table_name}"
                
                snowpark_df = self.session.create_dataframe(metadata_df)
                snowpark_df.write.save_as_table(full_name, mode='append')
                
                print(f"Saved metadata to: {full_name}")
                
            finally:
                monitor.end()

        def ensure_scoring_tables(self) -> None:
            """Ensure scoring-specific tables exist"""
            schemas = {
                'SCORES': f"""
                    CREATE TABLE IF NOT EXISTS {TABLE_NAMES['SCORES']} (
                        ID VARCHAR,
                        PROBABILITY FLOAT,
                        AFFINITY_SCORE FLOAT,
                        AFFINITY_GRADE VARCHAR,
                        SCORED_DATE TIMESTAMP_NTZ
                    )
                """,
                'METADATA': f"""
                    CREATE TABLE IF NOT EXISTS {TABLE_NAMES['METADATA']} (
                        SCORING_RUN_ID VARCHAR,
                        MODEL_NAME VARCHAR,
                        FEATURE_SAMPLE VARCHAR,
                        N_FEATURES NUMBER,
                        F1_SCORE FLOAT,
                        ROC_AUC FLOAT,
                        RECORDS_SCORED NUMBER,
                        SCORING_START TIMESTAMP_NTZ,
                        WHERE_CLAUSE VARCHAR,
                        GRADE_DISTRIBUTION VARIANT,
                        FEATURES_USED VARIANT
                    )
                """
            }
            
            for table_type, create_sql in schemas.items():
                table_name = TABLE_NAMES[table_type]
                full_name = f"{self.config.get_schema_db_name()}.{table_name}"
                
                # Check if table exists
                exists_query = f"""
                SELECT 1 
                FROM INFORMATION_SCHEMA.TABLES 
                WHERE TABLE_SCHEMA = '{self.config.SCHEMA}'
                AND TABLE_NAME = '{table_name}'
                """
        
                result = self.execute_with_retry(
                    'check_table_exists',
                    lambda: self.session.sql(exists_query).collect()
                )
                
                if not result:
                    print(f"Creating table: {table_name}")
                    self.execute_with_retry(
                        'create_table',
                        lambda: self.session.sql(create_sql).collect()
                    )



finally:
    monitor.end()
dep_checker.register_cell('snowflake_utils')

In [ ]:
# Cell: model_scoring
monitor.start('model_scoring')
try:
    class AffinityScorer:
        def __init__(self, config: EnvironmentConfig):
            self.config = config
            self.db_manager = ScoringDatabaseManager(config)
            self.session = config.session  # Ensure session is assigned
            self.timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            self.model = None
            self.features = None
            self.scaler = None
            self.imputers = None
            self.model_info = None
            # Ensure required tables exist
            self.db_manager.ensure_scoring_tables()

        def deserialize_object(self, serialized_str: str) -> Any:
            """Deserialize object using joblib"""
            import joblib
            import io
            buffer = io.BytesIO(base64.b64decode(serialized_str))
            return joblib.load(buffer)
         
        def get_best_model_info(self, target: str = 'COMMIT_MAJOR') -> Dict[str, Any]:
            print("\nDEBUG: Looking for best model...")
            full_table_name = f"{self.config.get_schema_db_name()}.{TABLE_NAMES['MODEL_RESULTS']}"
            print(f"Checking table: {full_table_name}")
            print("\nDEBUG: Table Information:")
            print(f"Full qualified table name: {self.config.get_schema_db_name()}.{TABLE_NAMES['MODEL_RESULTS']}")
            print("\nDEBUG: Checking Snowflake columns")

            columns_query = f"""
            SELECT *
            FROM {self.config.get_schema_db_name()}.{TABLE_NAMES['MODEL_RESULTS']}
            LIMIT 1
            """
            try:
                columns_result = self.db_manager.execute_with_retry(
                    'check_columns',
                    lambda: self.session.sql(columns_query).collect()
                )
                if columns_result:
                    print("Available columns:", list(columns_result[0].asDict().keys()))
                else:
                    print("No data found in table")
            except Exception as e:
                print(f"Error checking columns: {str(e)}")

            debug_query = f"""
            WITH MODEL_METRICS AS (
                SELECT
                    MODEL,
                    FEATURE_SAMPLE,
                    N_FEATURES,
                    F1_MEAN,
                    ROC_AUC_MEAN,
                    ACCURACY_MEAN,
                    PRECISION_MEAN,
                    RECALL_MEAN,
                    (F1_MEAN * {MODEL_SELECTION_WEIGHTS['PRIMARY']['F1_MEAN']} +
                     ROC_AUC_MEAN * {MODEL_SELECTION_WEIGHTS['PRIMARY']['ROC_AUC_MEAN']} +
                     ACCURACY_MEAN * {MODEL_SELECTION_WEIGHTS['PRIMARY']['BALANCED_ACCURACY_MEAN']})
                    AS PRIMARY_SCORE,
                    ROW_NUMBER() OVER (
                        PARTITION BY FEATURE_SAMPLE
                        ORDER BY F1_MEAN DESC
                    ) as RANK_IN_GROUP
                FROM {full_table_name}
                WHERE TARGET = '{target}'
            )
            SELECT *
            FROM MODEL_METRICS
            ORDER BY FEATURE_SAMPLE, RANK_IN_GROUP;
            """
            print("\nDEBUG: All Models and Rankings")
            print("=" * 80)
            debug_result = self.db_manager.execute_with_retry(
                'debug_models',
                lambda: self.session.sql(debug_query).to_pandas()
            )
            if not debug_result.empty:
                for feature_sample in debug_result['FEATURE_SAMPLE'].unique():
                    sample_data = debug_result[debug_result['FEATURE_SAMPLE'] == feature_sample]
                    print(f"\nFeature Sample: {feature_sample}")
                    print("-" * 40)
                    for _, row in sample_data.iterrows():
                        print(f"Model: {row['MODEL']}")
                        print(f"  Rank: {row['RANK_IN_GROUP']}")
                        print(f"  F1: {row['F1_MEAN']:.4f}")
                        print(f"  ROC-AUC: {row['ROC_AUC_MEAN']:.4f}")
                        print(f"  Primary Score: {row['PRIMARY_SCORE']:.4f}")
            else:
                print("No models found in debug query")

            query = f"""
            WITH MODEL_METRICS AS (
                SELECT
                    MODEL,
                    FEATURE_SAMPLE,
                    N_FEATURES,
                    F1_MEAN,
                    ROC_AUC_MEAN,
                    ACCURACY_MEAN,
                    PRECISION_MEAN,
                    RECALL_MEAN,
                    (F1_MEAN * {MODEL_SELECTION_WEIGHTS['PRIMARY']['F1_MEAN']} +
                     ROC_AUC_MEAN * {MODEL_SELECTION_WEIGHTS['PRIMARY']['ROC_AUC_MEAN']} +
                     ACCURACY_MEAN * {MODEL_SELECTION_WEIGHTS['PRIMARY']['BALANCED_ACCURACY_MEAN']})
                    AS PRIMARY_SCORE,
                    ROW_NUMBER() OVER (
                        PARTITION BY FEATURE_SAMPLE
                        ORDER BY F1_MEAN DESC
                    ) as rank_in_group
                FROM {full_table_name}
                WHERE TARGET = '{target}'
            )
            SELECT
                MODEL,
                FEATURE_SAMPLE,
                N_FEATURES,
                F1_MEAN,
                ROC_AUC_MEAN,
                ACCURACY_MEAN as BALANCED_ACCURACY_MEAN,
                PRECISION_MEAN,
                RECALL_MEAN,
                PRIMARY_SCORE
            FROM MODEL_METRICS
            WHERE rank_in_group = 1
            ORDER BY PRIMARY_SCORE DESC
            LIMIT 1;
            """
            result = self.db_manager.execute_with_retry(
                'get_best_model',
                lambda: self.session.sql(query).to_pandas()
            )
            if result.empty:
                raise ValueError("No model results found")
            self.model_info = result.iloc[0].to_dict()
            print("\nDEBUG: Selected Best Model")
            print("=" * 80)
            print(f"Model Name: {self.model_info['MODEL']}")
            print(f"Feature Sample: {self.model_info['FEATURE_SAMPLE']}")
            print(f"Number of Features: {self.model_info['N_FEATURES']}")
            print("\nPerformance Metrics:")
            print(f"F1 Score: {self.model_info['F1_MEAN']:.4f}")
            print(f"ROC-AUC: {self.model_info['ROC_AUC_MEAN']:.4f}")
            print(f"Balanced Accuracy: {self.model_info['BALANCED_ACCURACY_MEAN']:.4f}")
            return self.model_info

        def load_model_components(self) -> None:
            """Load model and its components"""
            query = f"""
            SELECT MODEL_OBJECT, SELECTED_FEATURES, METRICS, SCALER, IMPUTERS
            FROM {self.config.get_schema_db_name()}.{TABLE_NAMES['MODELS']}
            WHERE MODEL = '{self.model_info['MODEL']}'
            AND FEATURE_SAMPLE = '{self.model_info['FEATURE_SAMPLE']}'
            ORDER BY CREATED_AT DESC
            LIMIT 1;
            """
            print("\nLoading best model components...")
            result = self.db_manager.execute_with_retry(
                'load_model',
                lambda: self.session.sql(query).collect()
            )
            if not result:
                raise ValueError("Could not load model components")
        
            row = result[0]
        
            # Load base components
#            self.model = pickle.loads(base64.b64decode(row['MODEL_OBJECT']))
#            saved_features = pickle.loads(base64.b64decode(row['SELECTED_FEATURES']))
#            original_scaler = pickle.loads(base64.b64decode(row['SCALER']))
#            original_imputers = pickle.loads(base64.b64decode(row['IMPUTERS']))
            # NEW:
            self.model = self.deserialize_object(row['MODEL_OBJECT'])  
            saved_features = self.deserialize_object(row['SELECTED_FEATURES'])
            original_scaler = self.deserialize_object(row['SCALER'])
            original_imputers = self.deserialize_object(row['IMPUTERS'])            
        
            print(f"Loaded model type: {type(self.model).__name__}")
            print(f"Saved features from database: {saved_features}")
            
            # Determine the correct feature list to use
            model_features = None
            if hasattr(self.model, 'feature_names_in_') and self.model.feature_names_in_ is not None:
                model_features = self.model.feature_names_in_.tolist()
                print(f"Model's actual features: {model_features}")
            
            # Use model features if available, otherwise use saved features
            if model_features is not None:
                self.features = model_features
                print(f"Using model's feature list ({len(self.features)} features)")
                
                # Check for discrepancies with saved features
                if saved_features is not None:
                    saved_set = set(saved_features)
                    model_set = set(self.features)
                    if saved_set != model_set:
                        print(f"WARNING: Feature mismatch detected!")
                        print(f"Features in saved list but not in model: {saved_set - model_set}")
                        print(f"Features in model but not in saved list: {model_set - saved_set}")
                else:
                    print("WARNING: Saved features from database are None")
                    
            elif saved_features is not None:
                self.features = saved_features
                print(f"Using saved features from database ({len(self.features)} features)")
            else:
                raise ValueError("Both model features and saved features are None - cannot proceed")
            
            print(f"Final feature list: {self.features}")
        
            # Create dummy data with correct features
            if self.features is None or len(self.features) == 0:
                raise ValueError("No features available for model")
                
            dummy_data = pd.DataFrame(columns=self.features)
            if dummy_data.empty:
                print("Dummy data is empty. Initializing with zeros.")
                dummy_data = pd.DataFrame(np.zeros((2, len(self.features))), columns=self.features)
            print(f"Initialized dummy data with {len(dummy_data.columns)} columns")
        
            if isinstance(self.model, (RandomForestClassifier, LogisticRegression, DecisionTreeClassifier, SVC)):
                print("Handling specifics for RandomForest, LogisticRegression, DecisionTree, and SVC.")
                if hasattr(original_scaler, 'feature_names_in_') and original_scaler.feature_names_in_ is not None:
                    # Only check features that are actually in our model
                    scaler_features = set(original_scaler.feature_names_in_)
                    model_features_set = set(self.features)
                    common_features = model_features_set & scaler_features
                    print(f"Common features between model and scaler: {len(common_features)} out of {len(self.features)}")
                    
                    if len(common_features) != len(self.features):
                        print("WARNING: Not all model features are in the scaler")
                        
                print("Scaler feature names consistency check passed.")
                print("\nProcessing scaler:")
                if hasattr(original_scaler, 'mean_') and original_scaler.mean_ is not None:
                    for i, feature in enumerate(self.features):
                        if hasattr(original_scaler, 'feature_names_in_') and original_scaler.feature_names_in_ is not None:
                            if feature in original_scaler.feature_names_in_:
                                idx = list(original_scaler.feature_names_in_).index(feature)
                                dummy_data.iloc[:, i] = original_scaler.mean_[idx]
                                dummy_data.iloc[1, i] += original_scaler.scale_[idx]
                            else:
                                print(f"WARNING: Feature {feature} not found in original scaler, using zero")
                                dummy_data.iloc[:, i] = 0.0
                        else:
                            print(f"WARNING: Original scaler has no feature names, using zero for {feature}")
                            dummy_data.iloc[:, i] = 0.0
                else:
                    print("No mean or scale attributes found in the scaler.")
                    
                self.scaler = StandardScaler()
                self.scaler.fit(dummy_data)
                print("Scaler fitted with dummy data.")
        
            elif isinstance(self.model, MLPClassifier):
                print("Handling specifics for MLPClassifier.")
                # For MLPClassifier, we don't need to check dummy_data columns since we're using NoOpScaler
                class NoOpScaler:
                    def fit(self, X): return self
                    def transform(self, X): return X
                self.scaler = NoOpScaler()
                print("Set NoOpScaler for MLPClassifier.")
            else:
                raise ValueError(f"Unsupported model type: {type(self.model).__name__}")
        
            # Process imputers
            self.imputers = {}
            if original_imputers is not None:
                for strategy, imputer in original_imputers.items():
                    if hasattr(imputer, 'feature_names_in_') and imputer.feature_names_in_ is not None:
                        # Only use features that are both in the imputer AND in our model
                        valid_features = [f for f in imputer.feature_names_in_ if f in self.features]
                        if valid_features:
                            print(f"Features to keep for imputation ({strategy}): {valid_features}")
                            new_imputer = SimpleImputer(strategy=imputer.strategy)
                            indices = [list(imputer.feature_names_in_).index(f) for f in valid_features]
                            feature_values = imputer.statistics_[indices]
                            dummy_data_impute = pd.DataFrame(np.zeros((2, len(valid_features))), columns=valid_features)
                            dummy_data_impute.iloc[0] = feature_values
                            new_imputer.fit(dummy_data_impute)
                            self.imputers[strategy] = new_imputer
                            print(f"{strategy} imputer fitted with valid features.")
                        else:
                            print(f"WARNING: No valid features found for {strategy} imputer")
                    else:
                        print(f"WARNING: {strategy} imputer has no feature names")
            else:
                print("WARNING: No imputers found in saved model")
                
            print("Model components loaded successfully.")
            print(f"Final feature count: {len(self.features)}")
            print(f"Final imputer count: {len(self.imputers)}")
            

        def calculate_affinity_score(self, probabilities: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
            """Convert probabilities to scores and grades"""
            print("\nProbability Analysis:")
            print(f"Min probability: {probabilities.min():.3f}")
            print(f"Max probability: {probabilities.max():.3f}")
            print(f"Mean probability: {probabilities.mean():.3f}")
            
            probabilities = np.round(probabilities, 3)
            scores = np.round(probabilities * 100, 1)
            grades = np.full(len(scores), 'E', dtype='object')
            
            if GRADING_METHOD == 'PERCENTILE':
                print(f"\nUsing Percentile-Based Grading: {PERCENTILE_GRADE_CONFIG}")
                
                # Calculate percentile thresholds
                percentiles = {
                    grade: np.percentile(scores, pct) 
                    for grade, pct in PERCENTILE_GRADE_CONFIG.items() 
                    if grade != 'E'
                }
                
                print(f"Calculated thresholds: {percentiles}")

                # **ADD DEBUG: Show actual threshold values**
                print(f"Debug - Score thresholds:")
                print(f"A threshold (99th percentile): {percentiles['A']:.3f}")
                print(f"B threshold (95th percentile): {percentiles['B']:.3f}")
                print(f"C threshold (90th percentile): {percentiles['C']:.3f}")
                print(f"D threshold (80th percentile): {percentiles['D']:.3f}")

                
                # Assign grades (order matters - start with highest)
                grades[scores >= percentiles['A']] = 'A'
                grades[(scores >= percentiles['B']) & (scores < percentiles['A'])] = 'B'
                grades[(scores >= percentiles['C']) & (scores < percentiles['B'])] = 'C'
                grades[(scores > percentiles['D']) & (scores < percentiles['C'])] = 'D'
                # E is default

                # **ADD: Print final distribution (was missing!)**
                print("\nGrade Distribution:")
                for grade in ['A', 'B', 'C', 'D', 'E']:
                    count = np.sum(grades == grade)
                    print(f"Grade {grade}: {count} records ({count/len(grades):.1%})")

            
            else:  # FIXED method
                print(f"\nUsing Fixed Thresholds: {FIXED_GRADE_RANGES}")
                for grade, lower_bound, upper_bound in FIXED_GRADE_RANGES:

                    if grade == 'E':
                        mask = (scores < upper_bound)
                    elif grade == 'A':
                        mask = (scores >= lower_bound)
                    else:
                        mask = (scores >= lower_bound) & (scores < upper_bound)
                    grades[mask] = grade
                    count = np.sum(mask)
                    print(f"\nGrade {grade} ({lower_bound:.3f} to {upper_bound:.3f}):")
                    print(f"Found {count} records ({count/len(grades):.2%})")
                    if count > 0:
                        sample_idx = np.where(mask)[0][:5]
                        print("Sample records:")
                        for idx in sample_idx:
                            print(f"  Score: {scores[idx]:.3f}, Probability: {probabilities[idx]:.3f}")
                print("\nValidation Check:")
                for grade, lower_bound, upper_bound in FIXED_GRADE_RANGES:
                    if grade == 'E':
                        expected_mask = (scores < upper_bound)
                    elif grade == 'A':
                        expected_mask = (scores >= lower_bound)
                    else:
                        expected_mask = (scores >= lower_bound) & (scores < upper_bound)
                    actual_mask = (grades == grade)
                    mismatches = np.sum(expected_mask != actual_mask)
                    if mismatches > 0:
                        print(f"WARNING: Found {mismatches} mismatches for grade {grade}")
                        mismatch_idx = np.where(expected_mask != actual_mask)[0][:5]
                        print("Sample mismatches:")
                        for idx in mismatch_idx:
                            print(f"  Score: {scores[idx]:.1f}, Expected: {grade}, Got: {grades[idx]}")
                    else:
                        print(f"✓ Grade {grade}: No mismatches")
                print("\nValidation - Probability to Score Conversion:")
                for i in range(5):
                    idx = np.random.randint(0, len(probabilities))
                    expected = probabilities[idx] * 100
                    actual = scores[idx]
                    print(f"Probability: {probabilities[idx]:.3f} → Expected: {expected:.3f} → Actual Score: {actual:.1f}")
                
                        
                # Print final distribution
                print("\nGrade Distribution:")
                for grade in ['A', 'B', 'C', 'D', 'E']:
                    count = np.sum(grades == grade)
                    print(f"Grade {grade}: {count} records ({count/len(grades):.1%})")
            
            return scores, grades
        
            
        def calculate_affinity_score_OG(self, probabilities: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
            """Convert probabilities to scores and grades"""
            print("\nProbability Analysis:")
            print(f"Min probability: {probabilities.min():.3f}")
            print(f"Max probability: {probabilities.max():.3f}")
            print(f"Mean probability: {probabilities.mean():.3f}")

            probabilities = np.round(probabilities, 3)
            scores = np.round(probabilities * 100, 1)
            grades = np.full(len(scores), 'E', dtype='object')
            grade_ranges = GRADE_RANGES

            print("\nGrade Assignment Process:")
            for grade, lower_bound, upper_bound in grade_ranges:
                if grade == 'E':
                    mask = (scores < upper_bound)
                elif grade == 'A':
                    mask = (scores >= lower_bound)
                else:
                    mask = (scores >= lower_bound) & (scores < upper_bound)
                grades[mask] = grade
                count = np.sum(mask)
                print(f"\nGrade {grade} ({lower_bound:.3f} to {upper_bound:.3f}):")
                print(f"Found {count} records ({count/len(grades):.2%})")
                if count > 0:
                    sample_idx = np.where(mask)[0][:5]
                    print("Sample records:")
                    for idx in sample_idx:
                        print(f"  Score: {scores[idx]:.3f}, Probability: {probabilities[idx]:.3f}")

            print("\nValidation Check:")
            for grade, lower_bound, upper_bound in grade_ranges:
                if grade == 'E':
                    expected_mask = (scores < upper_bound)
                elif grade == 'A':
                    expected_mask = (scores >= lower_bound)
                else:
                    expected_mask = (scores >= lower_bound) & (scores < upper_bound)
                actual_mask = (grades == grade)
                mismatches = np.sum(expected_mask != actual_mask)
                if mismatches > 0:
                    print(f"WARNING: Found {mismatches} mismatches for grade {grade}")
                    mismatch_idx = np.where(expected_mask != actual_mask)[0][:5]
                    print("Sample mismatches:")
                    for idx in mismatch_idx:
                        print(f"  Score: {scores[idx]:.1f}, Expected: {grade}, Got: {grades[idx]}")
                else:
                    print(f"✓ Grade {grade}: No mismatches")

            print("\nValidation - Probability to Score Conversion:")
            for i in range(5):
                idx = np.random.randint(0, len(probabilities))
                expected = probabilities[idx] * 100
                actual = scores[idx]
                print(f"Probability: {probabilities[idx]:.3f} → Expected: {expected:.3f} → Actual Score: {actual:.1f}")

            return scores, grades

        def prepare_data(self, df: pd.DataFrame) -> pd.DataFrame:
            """Prepare data for scoring using saved feature order"""
            monitor.start('prepare_data')
            try:
                if 'DONOR_ID' not in df.columns:
                    raise ValueError("DONOR_ID not found in input data")
                donor_ids = df['DONOR_ID'].copy()
                print(f"Preserved {len(donor_ids)} DONOR_IDs")

                feature_df = df[self.features].copy()
                print(f"\nInitial feature columns: {feature_df.columns.tolist()}")

                print("\nPre-imputation validation:")
                null_counts = feature_df.isnull().sum()
                if null_counts.any():
                    print("Columns with nulls:")
                    print(null_counts[null_counts > 0])

                for strategy, imputer in self.imputers.items():
                    print(f"\nProcessing {strategy} imputation:")
                    features_to_impute = imputer.feature_names_in_
                    print(f"Imputer features: {features_to_impute.tolist()}")
                    if len(features_to_impute) > 0:
                        impute_df = feature_df[features_to_impute]
                        print(f"Data shape for imputation: {impute_df.shape}")
                        if impute_df.shape[1] != len(features_to_impute):
                            raise ValueError(f"Feature count mismatch: got {impute_df.shape[1]}, expected {len(features_to_impute)}")
                        feature_df[features_to_impute] = imputer.transform(impute_df)

                post_null_counts = feature_df.isnull().sum()
                if post_null_counts.any():
                    print("\nWARNING: Nulls remain after imputation:")
                    print(post_null_counts[post_null_counts > 0])

                if self.scaler is not None:
                    print(f"\nApplying scaler to features: {self.features}")
                    feature_df = pd.DataFrame(
                        self.scaler.transform(feature_df),
                        columns=self.features,
                        index=feature_df.index
                    )
                    print("\nApplied scaler to data.")

                feature_df['DONOR_ID'] = donor_ids
                print("\nPrepared feature columns for prediction:", feature_df.columns.tolist())
                return feature_df
            finally:
                monitor.end()

        def score_dataset(self, where_clause: Optional[str] = None,
                          save_results: bool = True,
                          limit: Optional[int] = None) -> pd.DataFrame:
            """Score the dataset"""
            monitor.start('score_dataset')
            try:
                print("Loading model components...")
                self.get_best_model_info()
                self.load_model_components()

                print("Loading data...")
                df = self.db_manager.load_data_for_scoring(
                    features=self.features, 
                    where_clause=where_clause,
                    limit=limit
                )
                print(f"Total records to process: {len(df)}")

                try:
                    feature_df = self.prepare_data(df)
                    donor_ids = feature_df['DONOR_ID']
                    prediction_features = feature_df.drop('DONOR_ID', axis=1)

                    print("\nFeature Statistics Before Prediction:")
                    print(prediction_features.describe())
                    print("\nFeature columns for prediction:", prediction_features.columns.tolist())

                    print("\nModel Information:")
                    print(f"Model type: {type(self.model).__name__}")
                    if hasattr(self.model, 'feature_names_in_'):
                        print(f"Model expected features: {self.model.feature_names_in_.tolist()}")

                    missing_features = set(self.model.feature_names_in_).difference(prediction_features.columns)
                    if missing_features:
                        print(f"Missing features at prediction time: {missing_features}")
                    else:
                        print("All expected features present at prediction time.")

                    probabilities = self.model.predict_proba(prediction_features)
                    print("\nRaw Prediction Output:")
                    print(f"Shape: {probabilities.shape}")
                    print(f"Class probabilities min: {probabilities.min():.3f}")
                    print(f"Class probabilities max: {probabilities.max():.3f}")

                    probabilities = np.round(probabilities[:, 1], 3)
                    print("\nVerification of probability rounding:")
                    print(f"Sample of raw vs rounded probabilities:")
                    sample_indices = np.random.choice(len(probabilities), min(5, len(probabilities)), replace=False)
                    for idx in sample_indices:
                        orig = probabilities[idx]
                        score = orig * 100
                        print(f"Probability: {orig:.3f} -> Score: {score:.1f}")

                    print("\nUnique probability values:")
                    unique_probs = np.unique(probabilities)
                    print("First few:", unique_probs[:10])
                    print("Last few:", unique_probs[-10:])

                    scores, grades = self.calculate_affinity_score(probabilities)

                    results_df = pd.DataFrame({
                        'ID': donor_ids,
                        'PROBABILITY': np.round(probabilities, 3),
                        'AFFINITY_SCORE': np.round(scores, 1),
                        'AFFINITY_GRADE': grades,
                        'SCORED_DATE': datetime.now()
                    })

                    print(f"\nFinal Results Shape: {results_df.shape}")
                    print("\nGrade Distribution:")
                    print(results_df['AFFINITY_GRADE'].value_counts())

                    if save_results:
                        print(f"\nSaving results...")
                        self.db_manager.save_scores(results_df)

                        metadata = {
                            'SCORING_RUN_ID': self.timestamp,
                            'MODEL': self.model_info['MODEL'],
                            'FEATURE_SAMPLE': self.model_info['FEATURE_SAMPLE'],
                            'N_FEATURES': self.model_info['N_FEATURES'],
                            'F1_SCORE': self.model_info['F1_MEAN'],
                            'ROC_AUC': self.model_info['ROC_AUC_MEAN'],
                            'RECORDS_SCORED': len(results_df),
                            'SCORING_START': datetime.now(),
                            'WHERE_CLAUSE': where_clause,
                            'GRADE_DISTRIBUTION': results_df['AFFINITY_GRADE'].value_counts().to_dict(),
                            'FEATURES_USED': self.features
                        }
                        self.db_manager.save_scoring_metadata(metadata)
                        print("Results and metadata saved successfully")
                    else:
                        print("\nResults not saved (save_results=False)")

                    return results_df

                except Exception as e:
                    print(f"Error during scoring: {str(e)}")
                    raise

            finally:
                monitor.end()

    # Test scoring
    if __name__ == "__main__":
        try:
            config = EnvironmentConfig()
            session = get_active_session()
            config.set_session(session)

            scorer = AffinityScorer(config)
            exclusion_where_clause = """
            DONOR_ID NOT IN (
                SELECT DISTINCT C."ucinn_ascendv2__Donor_ID__c"
                FROM PRE_PRODUCTION.ASCEND."Contact" C
                WHERE C."IsDeleted" = FALSE
                AND (
                    C."uff_Is_Lump_Sum_Donor__c" = TRUE
                    OR C."ucinn_ascendv2__First_and_Last_Name_Formula__c" LIKE ANY ('%Cash Donations%', '%Anonymous Donor%', 'DONOR%')
                    OR C."ucinn_ascendv2__Contact_Type__c" LIKE ANY ('%Estate Rep%', '%External Contact%', 'Student')
                    OR C."ucinn_ascendv2__Primary_Contact_Type__c" = 'Student'
                    OR C."ucinn_ascendv2__Is_Deceased__c" = TRUE
                    OR C."uff_UF_Disqualified__c" = TRUE
                )
            )
            """
            # Score with exclusion criteria  
            test_where = "1=1"
           # change where clause to random if you want a sample
            # "RANDOM() < 0.01"  # 1% sample for testing
            results = scorer.score_dataset(
                where_clause=test_where,
                save_results=True, # Set to False to test without saving
#                limit=LIMIT # 750000  # Explicitly set limit
            )
            # Show distribution
            print("\nScore Distribution:")
            print(results['AFFINITY_GRADE'].value_counts(normalize=True))
        except Exception as e:
            print(f"Error in scoring: {str(e)}")
            traceback.print_exc()
finally:
    monitor.end()
dep_checker.register_cell('model_scoring')

In [ ]:
 # Cell: scoring_report
monitor.start('scoring_report')
try:
    class ScoringReporter:
        def __init__(self, scoring_run_id: str, config: EnvironmentConfig):
            self.scoring_run_id = scoring_run_id
            self.db_manager = ScoringDatabaseManager(config)
            self.metadata = None
            self.load_metadata()

        def load_metadata(self) -> None:
            """Load metadata for scoring run"""
            query = f"""
            SELECT *
            FROM {TABLE_NAMES['METADATA']}
            WHERE SCORING_RUN_ID = '{self.scoring_run_id}'
            """
            result = self.db_manager.execute_with_retry(
                'load_metadata',
                lambda: self.db_manager.session.sql(query).collect()
            ).to_pandas()
            
            if result.empty:
                raise ValueError(f"No metadata found for run {self.scoring_run_id}")
            
            self.metadata = result.iloc[0].to_dict()

        def generate_distribution_analysis(self) -> pd.DataFrame:
            """Analyze score and grade distribution"""
            query = f"""
            SELECT 
                AFFINITY_GRADE,
                COUNT(*) as COUNT,
                COUNT(*) / SUM(COUNT(*)) OVER () as PERCENTAGE,
                MIN(AFFINITY_SCORE) as MIN_SCORE,
                MAX(AFFINITY_SCORE) as MAX_SCORE,
                AVG(AFFINITY_SCORE) as AVG_SCORE,
                PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY AFFINITY_SCORE) as Q1,
                PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY AFFINITY_SCORE) as MEDIAN,
                PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY AFFINITY_SCORE) as Q3
            FROM {TABLE_NAMES['SCORES']}_{self.scoring_run_id}
            GROUP BY AFFINITY_GRADE
            ORDER BY AFFINITY_GRADE
            """
            return self.db_manager.execute_with_retry(
                'distribution_analysis',
                lambda: self.db_manager.session.sql(query).collect()
            ).to_pandas()

        def generate_feature_importance_summary(self) -> pd.DataFrame:
            """Summarize feature importance for the model used"""
            query = f"""
            SELECT 
                FEATURE_NAME,
                IMPORTANCE,
                RANK() OVER (ORDER BY IMPORTANCE DESC) as IMPORTANCE_RANK
            FROM {TABLE_NAMES['FEATURES']}
            WHERE MODEL_NAME = '{self.metadata['MODEL_NAME']}'
            AND FEATURE_SAMPLE = '{self.metadata['FEATURE_SAMPLE']}'
            ORDER BY IMPORTANCE DESC
            LIMIT 20
            """
            return self.db_manager.execute_with_retry(
                'feature_importance',
                lambda: self.db_manager.session.sql(query).collect()
            ).to_pandas()

        def plot_distribution(self, dist_data: pd.DataFrame) -> None:
            """Plot score distribution"""
            plt.figure(figsize=(15, 10))
            
            # Create subplots
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
            
            # Plot 1: Grade Distribution
            sns.barplot(
                data=dist_data,
                x='AFFINITY_GRADE',
                y='PERCENTAGE',
                ax=ax1
            )
            ax1.set_title('Grade Distribution')
            ax1.set_ylabel('Percentage')
            
            # Add percentage labels
            for i, v in enumerate(dist_data['PERCENTAGE']):
                ax1.text(i, v, f'{v:.1%}', ha='center', va='bottom')
            
            # Plot 2: Box Plot of Scores by Grade
            sns.boxplot(
                data=dist_data,
                x='AFFINITY_GRADE',
                y='AVG_SCORE',
                ax=ax2
            )
            ax2.set_title('Score Distribution by Grade')
            ax2.set_ylabel('Score')
            
            plt.tight_layout()
            plt.show()

        def generate_report(self) -> None:
            """Generate comprehensive scoring report"""
            monitor.start('generate_report')
            try:
                print("\nAffinity Score Report")
                print("=" * 50)
                
                # Metadata
                print("\nScoring Run Information:")
                print(f"Run ID: {self.scoring_run_id}")
                print(f"Model: {self.metadata['MODEL_NAME']}")
                print(f"Feature Sample: {self.metadata['FEATURE_SAMPLE']}")
                print(f"Features Used: {self.metadata['N_FEATURES']}")
                print(f"Records Scored: {self.metadata['RECORDS_SCORED']:,}")
                print(f"Model Performance (F1): {self.metadata['F1_SCORE']:.4f}")
                print(f"Model Performance (ROC-AUC): {self.metadata['ROC_AUC']:.4f}")
                
                # Distribution Analysis
                print("\nScore Distribution Analysis:")
                dist_data = self.generate_distribution_analysis()
                print("\nGrade Distribution:")
                print("-" * 80)
                print(dist_data[['AFFINITY_GRADE', 'COUNT', 'PERCENTAGE', 
                               'MIN_SCORE', 'MAX_SCORE', 'AVG_SCORE']].to_string(index=False))
                
                # Plot distributions
                self.plot_distribution(dist_data)
                
                # Feature Importance
                print("\nTop Feature Importance:")
                print("-" * 80)
                feat_imp = self.generate_feature_importance_summary()
                print(feat_imp[['FEATURE_NAME', 'IMPORTANCE', 
                              'IMPORTANCE_RANK']].to_string(index=False))
                
                # Plot feature importance
                plt.figure(figsize=(12, 6))
                sns.barplot(
                    data=feat_imp.head(10),
                    x='IMPORTANCE',
                    y='FEATURE_NAME'
                )
                plt.title('Top 10 Feature Importance')
                plt.tight_layout()
                plt.show()
                
            finally:
                monitor.end()

    # Test reporting
#    if __name__ == "__main__":
# 
#        try:
#            config = EnvironmentConfig()
#            session = get_active_session()
#            config.set_session(session)
#            
#            # Initialize scorer
#            scorer = AffinityScorer(config)
#            
#            # Score a test sample
#            test_where = "RANDOM() < 0.01"  # 1% sample for testing
#            results = scorer.score_dataset(
#                where_clause=test_where,
#                save_results=False,  # Set to False to test without saving
#                limit=150000  # Add limit for testing
#            )
#            
#            # Show distribution
#            print("\nScore Distribution:")
#            print(results['AFFINITY_GRADE'].value_counts(normalize=True))
#            
#        except Exception as e:
#            print(f"Error in scoring: {str(e)}")
#            traceback.print_exc()

finally:
    monitor.end()
dep_checker.register_cell('scoring_report')

In [ ]:
 # Cell: score_validation
monitor.start('score_validation')
try:
    class ScoreValidator:
        def __init__(self, scoring_run_id: str, config: EnvironmentConfig):
            self.scoring_run_id = scoring_run_id
            self.db_manager = ScoringDatabaseManager(config)
            self.validation_results = {}
            
            # Ensure tables exist
            self.db_manager.ensure_scoring_tables()

        def validate_score_counts(self) -> bool:
            """Validate minimum number of scores generated"""
            query = f"""
            SELECT COUNT(*) as SCORE_COUNT
            FROM {TABLE_NAMES['SCORES']}
            """
            result = self.db_manager.execute_with_retry(
                'count_scores',
                lambda: self.db_manager.session.sql(query).to_pandas()
            )
            
            score_count = result['SCORE_COUNT'].iloc[0]
            is_valid = score_count >= VALIDATION_THRESHOLDS['min_score_count']
            
            self.validation_results['score_count'] = {
                'status': 'PASS' if is_valid else 'FAIL',
                'actual': score_count,
                'threshold': VALIDATION_THRESHOLDS['min_score_count']
            }
            
            return is_valid

        def validate_null_values(self) -> bool:
            """Validate percentage of null values"""
            query = f"""
            SELECT 
                COUNT(*) as TOTAL_RECORDS,
                COUNT(*) - COUNT(AFFINITY_SCORE) as NULL_SCORES,
                COUNT(*) - COUNT(AFFINITY_GRADE) as NULL_GRADES
            FROM {TABLE_NAMES['SCORES']}
            """
            result = self.db_manager.execute_with_retry(
                'check_nulls',
                lambda: self.db_manager.session.sql(query).to_pandas()
            )
            
            row = result.iloc[0]
            null_pct = max(
                row['NULL_SCORES'] / row['TOTAL_RECORDS'],
                row['NULL_GRADES'] / row['TOTAL_RECORDS']
            )
            
            is_valid = null_pct <= VALIDATION_THRESHOLDS['max_null_percentage']
            
            self.validation_results['null_values'] = {
                'status': 'PASS' if is_valid else 'FAIL',
                'actual': null_pct,
                'threshold': VALIDATION_THRESHOLDS['max_null_percentage']
            }
            
            return is_valid

        def validate_grade_distribution(self) -> bool:
            """Validate grade distribution is within expected bounds"""
            query = f"""
            SELECT 
                AFFINITY_GRADE,
                COUNT(*) / SUM(COUNT(*)) OVER () as grade_pct
            FROM {TABLE_NAMES['SCORES']}
            GROUP BY AFFINITY_GRADE
            ORDER BY AFFINITY_GRADE
            """
            result = self.db_manager.execute_with_retry(
                'grade_distribution',
                lambda: self.db_manager.session.sql(query).to_pandas()
            )
            
            grade_distribution = dict(zip(result['AFFINITY_GRADE'], result['GRADE_PCT']))
            all_valid = True
            
            self.validation_results['grade_distribution'] = {}
            
            for grade, (min_pct, max_pct) in VALIDATION_THRESHOLDS['grade_distribution_bounds'].items():
                actual_pct = grade_distribution.get(grade, 0)
                is_valid = min_pct <= actual_pct <= max_pct
                
                self.validation_results['grade_distribution'][grade] = {
                    'status': 'PASS' if is_valid else 'FAIL',
                    'actual': actual_pct,
                    'bounds': (min_pct, max_pct)
                }
                
                all_valid &= is_valid
            
            return all_valid

        def validate_metadata(self) -> bool:
            """Validate metadata was properly saved"""
            query = f"""
            SELECT *
            FROM {TABLE_NAMES['METADATA']}
            WHERE SCORING_RUN_ID = '{self.scoring_run_id}'
            """
            result = self.db_manager.execute_with_retry(
                'check_metadata',
                lambda: self.db_manager.session.sql(query).to_pandas()
            )
            
            is_valid = not result.empty
            
            self.validation_results['metadata'] = {
                'status': 'PASS' if is_valid else 'FAIL',
                'found': not result.empty
            }
            
            return is_valid

        def run_all_validations(self) -> bool:
            """Run all validations and return overall status"""
            monitor.start('run_validations')
            try:
                validations = [
                    ('Score Counts', self.validate_score_counts()),
                    ('Null Values', self.validate_null_values()),
#                    ('Grade Distribution', self.validate_grade_distribution()),
                    ('Metadata', self.validate_metadata())
                ]
                
                all_passed = all(result for _, result in validations)
                
                # Print validation results
                print("\nValidation Results:")
                print("=" * 50)
                
                for name, result in validations:
                    status = "✓" if result else "✗"
                    print(f"{status} {name}")
                
                if 'score_count' in self.validation_results:
                    print(f"\nScore Count: {self.validation_results['score_count']['actual']:,} "
                          f"(min: {self.validation_results['score_count']['threshold']:,})")
                
                if 'null_values' in self.validation_results:
                    print(f"Null Percentage: {self.validation_results['null_values']['actual']:.2%} "
                          f"(max: {self.validation_results['null_values']['threshold']:.2%})")
                
                return all_passed
            
            finally:
                monitor.end()

    # Test validation
    if __name__ == "__main__":
        try:
            config = EnvironmentConfig()
            session = get_active_session()
            config.set_session(session)
            
            # Get latest scoring run
            query = f"""
            SELECT SCORING_RUN_ID
            FROM {TABLE_NAMES['METADATA']}
            ORDER BY SCORING_START DESC
            LIMIT 1
            """
            
            result = ScoringDatabaseManager(config).execute_with_retry(
                'get_latest_run',
                lambda: session.sql(query).to_pandas())
            
            if not result.empty:
                scoring_run_id = result['SCORING_RUN_ID'].iloc[0]
                validator = ScoreValidator(scoring_run_id, config)
                all_passed = validator.run_all_validations()
                
                print(f"\nOverall Validation Status: {'PASS' if all_passed else 'FAIL'}")
            else:
                print("No scoring runs found to validate")
            
        except Exception as e:
            print(f"Error in validation: {str(e)}")
            traceback.print_exc()

finally:
    monitor.end()
dep_checker.register_cell('score_validation')